### Проверка работоспособности и качества пайплайна

In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import roc_auc_score, precision_recall_curve, confusion_matrix
import dill
dill._dill._reverse_typemap['ClassType'] = type

In [2]:
X_test = pd.read_csv("X_test.csv")
y_test = pd.read_csv("y_test.csv")

In [3]:
X_test.head(3)

,age,job,marital,education,default,balance,housing,loan,contact,duration,campaign,pdays,previous,poutcome
0,48,admin.,married,secondary,0,35,0,0,cellular,80,4,-1,0,0
1,46,technician,married,secondary,0,2527,1,1,unknown,201,3,-1,0,0
2,46,services,single,unknown,0,454,0,0,cellular,155,9,-1,0,0


In [4]:
with open('catboost_pipeline.dill', 'rb') as in_strm:
    pipeline = dill.load(in_strm)

Разделим данные на train и test и сохраним выборки на диск:

In [5]:
# прогнозы:
y_pred = pipeline.predict_proba(X_test)[:, 1]
pd.DataFrame({'preds': y_pred}).to_csv("test_predictions.csv", index=None)

In [6]:
# посчитаем метрики:
precision, recall, thresholds = precision_recall_curve(y_test, y_pred)
fscore = (2 * precision * recall) / (precision + recall)
roc_auc = roc_auc_score(y_test, y_pred)
ix = np.argmax(fscore)

print(f'thresholds={thresholds[ix]:.3f}, \nf1_score={fscore[ix]:.3f}, \nroc_auc={roc_auc:.3f}, \nrecall={recall[ix]:.3f}, \nprecision={precision[ix]:.3f}')

thresholds=0.276, 
f1_score=0.568, 
roc_auc=0.905, 
recall=0.654, 
precision=0.502


In [7]:
cnf_matrix = confusion_matrix(y_test, y_pred>thresholds[ix])
cnf_matrix

array([[9242,  810],
       [ 434,  817]], dtype=int64)